In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# PyBEAST Tutorial 1: Designing agents and running simulations

pyBEAST is an python implementation of Bioinspired Evolutionary Agent Simulation Toolkit (BEAST), an educational tool to help students to explore concepts of bio-inspired computing and game development. BEAST provides a modular framework that allows users to create and interact with simple objects and agents within a 2D environment. Agents could for example be representations of animals, robots, or other abstract objects.    

In BEAST, agents can be equiped with sensors that detect objects or other agents within their vicinity. Users can specify the control logic of these agents to determine how they respond to sensor stimuli. This flexibility allows users to design agents that display complex autonomous behaviours, such as obstacle avoidance, pathfinding and hunting.   

In this tutorial, you will learn how to design your own agent and implement its control logic to specify its movement during the simulation. By default, agents bn BEAST behave similar to a Braitenberg vehicle, named after the Italian cyberneticist Valentino Braitenberg. A Braitenberg vehicle has two wheels, each driven by its own motor. Typically, it is equipped with sensors whose inputs actuate the motors to generate locomotion. In such a scenario, the Braitenberg vehicle responds autonomously to the sensor input it receives from its environment. 

To make things simple, we start by designing a passive Braitenberg vehicle without sensors. In the next tutorial, you will learn how to incorporate sensors. So without further ado, let's design our own Braitenberg vehicle.          

## Braitenberg vehicle

To create a custom agent, we define a new class that inherits from the `core.agents.Animat` class, which serves as the base agent class in BEAST.

In [ ]:
from pybeast.core.agents.animat import Animat
from pybeast.core.world.world import WORLD_DISP_PARAM
from pybeast.core.utils.vector2D import Vector2D

class BraitenbergVehicle(Animat):

    def __init__(self):
    
        super().__init__(startLocation = Vector2D(400, 300), startOrientation=np.pi/2)
        self.SetMaxSpeed(100.0)
        self.SetMinSpeed(0.0)
        self.SetRadius(10.0)
    
    def Control(self):

        self.controls['left'] = 0.5
        self.controls['right'] = 0.5


In the constructor `BraitenbergVehicle.__init__`, we define the vehicle's start location and orientation and set values for its maximum speed, minimum speed, and radius. To control the vehicle's movement during the simulation we need to overwrite the ```Animat.Control``` method.  

Let's initialize a brand new Braitenberg vehicle  

In [ ]:
braiti = BraitenbergVehicle()
braiti.Init()

The current location and orientation on an animat can be accessed via

In [ ]:
braiti.location, braiti.orientation

BEAST implements its on vector class ```core.utils.vector2D.Vector2D```, which can be instantiated from an x and y coordinate  

In [ ]:
vector1 = Vector2D(500, 400)

The ```Vector2D``` class supports standard vector algebra operations like addition, subtraction and scalar multiplication 

In [ ]:
vector2 = Vector2D(200, 200) 
vector3 = vector1 - 2.0*vector2
vector3.x, vector3.y

By default, BEAST generates a rectangular 2D world. The width and height of the world can be specified via the `core.world.world.WORLD_DISP_PARAM` parameter, which defaults to                

In [ ]:
WORLD_DISP_PARAM.width, WORLD_DISP_PARAM.height

The origin (0, 0) of the coordinate system is located at the bottom-left corner of the world. Therefore, the x and y coordinate of an animat must be larger than 0 and smaller than the world's width and height, respectively. By default, the world has periodic boundaries, i.e. when an animat crosses one of the world boundaries it reappears on the opposite side. 

The x and y coordinate of the vehicle's current location   

In [ ]:
braiti.location.x, braiti.location.y  

are equivalent to its start location specified in ```BreitenbergVehicle.__init__```. 

To change the vehicle's position, we need to actuate its motors. By default, each animat has a left and a right wheel, each powered by its own motor. To actuate these motors, we set values in the `Animat.control` dictionary        

In [ ]:
braiti.controls

Control values for the left and right wheels should range from 0.0 to 1.0. With both controls set to zero, the animat moves straight at its minimum speed (zero by default). With both controls set to one, it moves straight at maximum speed. If the control values differ, the animat will turn in the direction of the larger control value. The minimum and maximum speeds can be accessed through the `minSpeed` and `maxSpeed` attributes.

In [ ]:
braiti.minSpeed, braiti.maxSpeed

The animat's control values during a simulation are set by the `Animat.Control` method, which is called during each time step of the simulation to update the values in the `Animat.controls` dictionary. Using these values and the animat's previous position and orientation, the animat's new position and orientation are calculated. For more details, refer to the implementation of the ```Animat.Update``` method. In our example, ```BraitenbergVehicle.Control``` sets both the value of the left and right controls to 0.5. Given our discussion, what kind of movement to you expect for these values? To verify, let's simulate the ```BraitenbergVehicle```. 

## Simulation


For this, we need to create a new class that inherits from ```core.simulation.Simulation``` class, which serves as a base class for simulations in BEAST.  

In [ ]:
from pybeast.core.simulation import Simulation  

class BraitenbergSimulation(Simulation):

    def __init__(self):
        
        super().__init__('Breitenberg')

        self.whatToLog['Generation'] = self.whatToLog['Assessment'] = self.whatToLog['Update'] = True 
        self.whatToSave['Simulation'] = self.whatToSave['Update'] = True 

        self.breiti = BraitenbergVehicle()

    def BeginAssessment(self):
        
        self.theWorld.Add(self.breiti)
        super().BeginAssessment()

    def CreateDataStructSimulation(self):
        
        self.locations, self.velocities, self.times = [], [], []

    def SaveUpdate(self):

        self.locations.append((self.breiti.location.x, self.breiti.location.y))
        self.velocities.append(self.breiti.velocity.GetLength())
        self.times.append(self.timeStep)        
        
        return 


In [ ]:
simulation = BraitenbergSimulation() 

In BEAST, simulations are structured into runs, generations, assessements and timesteps. After each assessment, the fitness score of each agent is recorded. The fitness scores are used by a genetic algorithm to generate the next generation of agents, which will be explained in the third tutorial.

Since our simple example does not involve evolution, we set number of runs, generations and assessments to one.  

In [ ]:
simulation.SetRuns(1)
simulation.SetGenerations(1)
simulation.SetAssessments(1)
simulation.SetTimeSteps(100) 

Each `Simulation` instance has an attribute ```theWorld``` which is an instance of the ```core.world.world.World``` class representing the world the agents live in    

In [ ]:
simulation.theWorld

Agents and objects must be added to the world at the start of each assessment, which is handeled by the `Simulation.BeginAssessment` method. In `BraitenbergSimulation.BeginAssessment`, we use the `World.Add` method to add the `breiti` vehicle the world using the  method.

To save simulation output, we need to create data containers and tell the simulation object to save the the output variables of interest. The ```Simulation``` class allows user to create data containers at the beginning of a simulation, run, generation and assessment, by implementing the ```Simulation.CreateDataStructure...``` methods. We can populate these containers with simulation data from individual runs, generations, assessments or timesteps by implementing the ```Simulation.Save...``` methods.      
In the `BraitenbergSimulation` class,  `CreateDataStructureSimulation` initializes two lists `self.locations` and `self.velocities`, to store the vehicle's location and velocity during the simulation. The ```BraitenbergSimulation.SaveUpdate``` method is called after each time step to append the vehicle's current location and velocity to the ```self.locations``` and ```self.velocities``` lists. 

Now, let's run the simulation and plot the vehicle's position and velocity over time. 

In [ ]:
simulation.RunSimulation(render=False)

To plot the position and velocity of the vehicle as a function of time, we access the data in the `locations` and `velocities` attributes

In [ ]:
x_arr = [vec[0] for vec in simulation.locations]
y_arr = [vec[1] for vec in simulation.locations]

plt.figure(figsize = (10, 5))
gs = plt.GridSpec(1, 2)
ax0 = plt.subplot(gs[0])
ax1 = plt.subplot(gs[1])

ax0.plot(x_arr, y_arr, '-')
ax0.plot(x_arr[0], y_arr[0], 'o', c='g')
ax0.plot(x_arr[-1], y_arr[-1], 'o', c='r')
ax0.set_xlabel('$x$')
ax0.set_ylabel('$y$')

ax1.plot(simulation.times, simulation.velocities)
ax1.set_xlabel(r'time')
ax1.set_ylabel('speed')

## Questions:

1. Think about how you could change the ```BraitenbergVehicle.Control``` so that the vehicle goes in a circle and not in a straight line.
2. How can you make the vehicle go counter clockwise or clockwise?
3. How can you make the radius of the cirle smaller/larger?

Rerun the simulation to confirm that your changes work as expected.

# Random control

In the current implementation of the ```BraitenbergVehicle.Control``` method,  the left and right control values are set to constant value. This implies that vehicle will always follow the same trajectory no matter how often we rerun the simulation. To make things a bit more intersting, let's add some randomness to the ```BraitenbergVehicle.Control``` method.      

In [ ]:
class BraitenbergVehicle(Animat):

    def __init__(self):
    
        super().__init__(startLocation = Vector2D(400, 300), startOrientation=np.pi/2)
        self.SetMaxSpeed(100.0)
        self.SetMinSpeed(0.0)
        self.SetRadius(10.0)
        self.lastTurn = 'left'
        self.controls['left'] = 0.6    
        self.controls['right'] = 0.4
    
    def Control(self):

        if np.random.rand() < 0.1:
            if self.lastTurn == 'left':
                self.controls['right'] = 0.6    
                self.controls['left'] = 0.4
                self.lastTurn = 'right'
            elif self.lastTurn == 'right':
                self.controls['left'] = 0.6
                self.controls['right'] = 0.4
                self.lastTurn = 'left'
            else:
                assert False

class BraitenbergSimulation(Simulation):

    def __init__(self):
        
        super().__init__('Breitenberg')

        self.Runs = 1
        self.generations = 1
        self.assessments = 5
        self.timeSteps = 250
        
        self.whatToLog['Generation'] = self.whatToLog['Assessment'] = self.whatToLog['Update'] = True 
        self.whatToSave['Simulation'] = self.whatToSave['Assessment'] = self.whatToSave['Update'] = True 

    def BeginAssessment(self):

        self.breiti = BraitenbergVehicle()
        self.theWorld.Add(self.breiti)
        super().BeginAssessment()

    def CreateDataStructSimulation(self):
        
        self.allLocations = []

    def CreateDataStructAssessment(self):

        self.locations = []

    def SaveAssessment(self):

        self.allLocations.append(self.locations.copy())

    def SaveUpdate(self):

        self.locations.append((self.breiti.location.x, self.breiti.location.y))


Note that we changed the number of assessments to 5 in the `BraitenbergSimulation.__init__`. To save the vehicle's position during each assessment, we create a list `self.locations` at the beginning of each asssesment by overwriting the `Simulation.CreateDataStructAssessment` method. At end of each assessment, we use the `SaveAssessment` method to append a copy of this list to `self.allLocations`, which is initialized at the beginning of the simulation when ```CreateDataStructSimulation``` is called. Let's run the simulation and plot the position of the vehicle during each assessment.

In [ ]:
simulation = BraitenbergSimulation() 
simulation.RunSimulation(render=False)

In [ ]:
plt.figure(figsize = (10, 5))

for location in simulation.allLocations:

    x_arr = [vec[0] for vec in location]
    y_arr = [vec[1] for vec in location]

    plt.plot(x_arr, y_arr, '-')
    plt.plot(x_arr[0], y_arr[0], 'o', c='k')
    plt.plot(x_arr[-1], y_arr[-1], 'o', c='k')

ax0.set_xlabel('$x$')
ax0.set_ylabel('$y$')

Why do some of the vehicle's trajectories contain straight vertical and horizontal lines? 

To visualize the simulation in real time, you can use the BEAST's GUI. To start the GUI set the `render` parameter in the `Simulation.RunSimulation` method to `True`. Select the start item in the Simulation drop-down menu to begin rendering the simulation. 

In [ ]:
simulation.RunSimulation(render=True)